# Анализ тональности отзывов

Сначала возьмем выборку отзывов на фильмы из NLTK. Получим ID-шники негативных и позитивных отзывов.

In [2]:
from nltk.corpus import movie_reviews
 
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

print negids[:5]

[u'neg/cv000_29416.txt', u'neg/cv001_19502.txt', u'neg/cv002_17424.txt', u'neg/cv003_12683.txt', u'neg/cv004_12641.txt']


Приготовим список текстов и классов как обучающую выборку. Получаем слова из отзыва, но слова получаем в листе. Поэтому, так как дальше CountVectorizer из sklearn будет работать с текстами все же, мы их снова джойним через пробелы.  Здесь мы получаем список текстов и список ответов. 0 будем использовать для негативного класса, 1 — для позитивного.

In [3]:
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]

texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [4]:
print texts[0]

plot : two teen couples go to a church party , drink and then drive . they get into an accident . one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . what ' s the deal ? watch the movie and " sorta " find out . . . critique : a mind - fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn ' t snag this one correctly . they seem to have taken this pretty neat concept , but executed it terribly . so what are the problems with the movie ? well , its main problem is that it ' s simply too jumbled . it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member , have no idea

Импортируем нужные модули: модули для построения признаков на текстах, разные классификаторы, линейные классификаторы, в частности линейный svm. Это модуль cross_validation для того, чтобы мы могли оценить качество, и модуль Pipeline, так будет несколько удобнее.

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline

### Оценка качества работы разных классификаторов

 Дальше мы пишем свою простенькую функцию, которая фактически возвращает Pipeline для текстовой классификации.

In [6]:
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

Сравним разные классификаторы на этом датасете. Сначала у нас используется CountVectorizer, затем TfidfTransformer, то есть мы на основе частот подсчитываем Tfidf-ы. После этого используем классификатор — тот, до которого дошла очередь в цикле. Нам не обязательно было разносить CountVectorizer и TfidfTransformer. Мы могли бы сразу использовать TfidfVectorizer, он тоже есть в sklearn. Но в таком виде будет несколько удобнее, если мы захотим здесь вместо TfidfTransformer использовать какой-то другой, ну например, как-нибудь понижать размерность.

In [7]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print clf
    print cross_val_score(text_classifier(CountVectorizer(), TfidfTransformer(), clf()), texts, labels).mean()
    print "\n"

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.813511115906


<class 'sklearn.svm.classes.LinearSVC'>
0.845507183831


<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
0.840010669352




### Подготовка классификатора, обученного на всех данных

Создаем pipeline из Vectorizer и Classifier. Сразу будем использовать TfidfVectorizer. Остается обучить.

In [8]:
clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer()),
            ("classifier", LinearSVC())]
        )


clf_pipeline.fit(texts, labels)

print clf_pipeline

Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])


 Ну и можно как-то его потестировать, например, поиграться с разными отзывами. Ну вот здесь вот были придуманы пара достаточно простых отзывов, они действительно очень ярко позитивные и негативные. Первый — позитивный, второй — негативный. Ну и как вы видите, результаты на них как раз такие, какие следовало ожидать. Но не стоит обольщаться: если получается придумать несколько хороших примеров, это отнюдь не значит, что классификатор хорош. Нужно всегда смотреть на качество.

In [9]:
print clf_pipeline.predict(["Amazing film! I will advice it to all my friends. Genious",
                           "Awful film! The man who advised me to watch it is really crazy idiot."])

[1 0]


## Понижение размерности и ансамбли деревьев

Попробуем доработать наш простой baseline, добавив некоторое преобразование признаков, попробовав встроить ансамбли деревьев. Для начала попробуем применить какие-нибудь матричные разложения. Попробуем посмотреть на качество и SVD, и NMF. Мы, как и ранее, делаем pipeline для текстовой классификации. Сначала получаем частоты слов, затем выполняем преобразования. Раньше это было TF-IDF-преобразование, теперь это SVD и NMF. И опять же берем всего десять компонент и на этом строим линейный SVM. Как вы видите, результаты не очень впечатляют. Они существенно ниже, чем те результаты, которые у нас были ранее. Ну, кстати, вот у NMF немножко получше. И здесь можно предположить, что что-то мы делаем неправильно. Можно попробовать сделать больше компонент. Давайте выставим количество компонент, равным тысяче, и посмотрим на результат. Вот результат уже здесь есть. И как вы помните, примерно такой результат был и у исходного baseline. То есть у нас получилось сделать такое преобразование, которое по крайней мере не испортило наш классификатор. Как правило, если преобразование признаков с помощью матричных разложений действительно позволяет как-то поднять качество, то количество компонент должно быть не таким большим, чтобы качество было совсем такое же, как изначально, и не таким маленьким, чтобы качество уже начало «проседать». Но в то же время, далеко не всегда такое количество компонент есть. Не всегда с помощью матричных разложений можно действительно улучшить качество. И не стоит в этом месте поддаваться иллюзиям, что если сделать что-то «умное» с признаками, то обязательно это поможет и повысит качество. С другой стороны, понизив размерность пространства признаков, мы можем построить более сложные классификаторы. Например, композиции деревьев. Ну, давайте попробуем посмотреть, что получается с RandomForest. Опять-таки мы считаем частоты, дальше делаем SVD-преобразование и затем запускаем обучение RandomForest. И как вы видите, результаты тоже достаточно плачевные. Конечно, это лучше, чем то, что мы видели до этого на десяти компонентах. Но здесь и компонент побольше, и как вы видите, лучше не стало. Можно предположить, что дело в количестве компонент. Надо сделать побольше компонент. Но оказывается, что больше компонент не всегда хорошо. Смотрите, сделали мы 1000 компонент, сделали и деревьев чуть побольше, и качество у нас еще дополнительно просело, то есть так тоже бывает. Ну, можно предположить, что проблема в том, что мы используем частоты слов, а если мы будем использовать, например, TF-IDF, получится более удачный классификатор. Это предположение в какой-то степени оправдывается, но, конечно, это, скорее, шутливый результат. Мы действительно чуть-чуть улучшили качество, но оно по-прежнему совершенно неприемлемое. Можно предположить, если не получается построить хороший классификатор на признаках, которые получаются после SVD-разложения, то, может быть, можно совместить эти признаки с теми признаками, которые были до этого. И уж тогда, по крайней мере, должно получиться не хуже. Ну, давайте посмотрим, что получается из этого. Давайте добавим просто одну компоненту из SVD-разложения. Мы воспользуемся FeatureUnion из sklearn.pipeline, которая позволит нам совместить эти преобразования и получать единое множество признаков. И вот давайте опять оценим качество. И, как вы видите, качество получается тоже не очень хорошее. Уж по крайней мере заметно ниже, чем без добавления одной компоненты из SVD. Таким образом, как вы видите наш baseline был действительно не так уж плох. И просто поделать какие-то логичные, какие-то достаточно сложные, с точки зрения математики, которая стоит за ними, вещи и поднять качество, довольно затруднительно.

In [23]:
%%time
from sklearn.decomposition import NMF, TruncatedSVD

v = CountVectorizer()
mx = v.fit_transform(texts)
mf = TruncatedSVD(10)
u = mf.fit_transform(mx)

CPU times: user 1.77 s, sys: 0 ns, total: 1.77 s
Wall time: 1.77 s


In [22]:
for transform in [TruncatedSVD, NMF]:
    print transform
    print cross_val_score(text_classifier(CountVectorizer(), transform(n_components=10), LinearSVC()), texts, labels).mean()
    print "\n"


<class 'sklearn.decomposition.truncated_svd.TruncatedSVD'>
0.569963676251


<class 'sklearn.decomposition.nmf.NMF'>
0.650514286742







Если задать n_components=1000:

In [12]:
%%time
print cross_val_score(text_classifier(TfidfVectorizer(), TruncatedSVD(n_components=1000), LinearSVC()),
                      texts, 
                      labels
                     ).mean()

0.842014169858
CPU times: user 3min 11s, sys: 13.5 s, total: 3min 25s
Wall time: 2min 24s


## Ансамбли деревьев на преобразованных признаках

In [14]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [15]:
%%time
print cross_val_score(
    Pipeline([
            ("vectorizer", CountVectorizer()),
            ("transformer", TruncatedSVD(100)),
            ("classifier", RandomForestClassifier(100))
        ]),
    texts,
    labels
    )

[ 0.70209581  0.71621622  0.69069069]
CPU times: user 13.9 s, sys: 600 ms, total: 14.5 s
Wall time: 14.3 s


Больше компонент и больше деревьев:

In [19]:
%%time
print cross_val_score(text_classifier(CountVectorizer(), TruncatedSVD(n_components=1000), RandomForestClassifier(1000)),
                      texts, 
                      labels
                     ).mean()

0.561485137832
CPU times: user 4min 43s, sys: 13.6 s, total: 4min 57s
Wall time: 3min 56s


Tf*Idf вместо частот слов:

In [18]:
%%time
print cross_val_score(text_classifier(TfidfVectorizer(), TruncatedSVD(n_components=1000), RandomForestClassifier(1000)),
                      texts, 
                      labels
                     ).mean()

0.590001678325
CPU times: user 4min 39s, sys: 14.3 s, total: 4min 53s
Wall time: 3min 52s


## Совмещаем Tf*Idf и SVD

In [16]:
from sklearn.pipeline import FeatureUnion

estimators = [('tfidf', TfidfTransformer()), ('svd', TruncatedSVD(1))]
combined = FeatureUnion(estimators)

In [17]:
%%time
print cross_val_score(
    Pipeline([
            ("vectorizer", CountVectorizer()),
            ("transformer", combined),
            ("classifier", LinearSVC())
        ]),
    texts,
    labels
    )

[ 0.74251497  0.78978979  0.62912913]
CPU times: user 10.7 s, sys: 24 ms, total: 10.7 s
Wall time: 10.7 s
